In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Install Required Packages

In [ ]:
!pip install groq pillow python-dotenv --quiet
!pip install openai


# Import Libraries

In [ ]:
import os
from groq import Groq
from PIL import Image, ImageDraw, ImageFont
import json


# Set API Key

In [ ]:
GROQ_API_KEY = "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"  # paste key here

client = Groq(api_key=GROQ_API_KEY)


In [ ]:
from groq import Groq
client = Groq(api_key="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")

for m in client.models.list().data:
    print(m.id)


# LLM Wrapper

In [ ]:

from groq import Groq

client = Groq(api_key="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")

def llm(prompt):
    try:
        completion = client.chat.completions.create(
            model="llama-3.3-70b-versatile",
            messages=[
                {"role": "user", "content": prompt}
            ]
        )
        return completion.choices[0].message.content
    except Exception as e:
        return f"[ERROR] {e}"




# Generate Video Ideas

In [ ]:

from groq import Groq

client = Groq(api_key="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")

def generate_video_ideas(topic):
    prompt = f"Generate 5 YouTube video ideas about: {topic}"

    response = client.chat.completions.create(
        model="llama-3.3-70b-versatile",   # UPDATED MODEL
        messages=[{"role": "user", "content": prompt}]
    )

    return response.choices[0].message.content

print("---- VIDEO IDEAS ----")
print(generate_video_ideas("cooking"))



# Pick an Idea and Generate Script

In [ ]:
chosen_idea = input("Choose one idea from the above list: ")

script = llm(f"""
Write a YouTube script for this video idea: {chosen_idea}
Keep it engaging, simple, and suitable for beginners.
""")

print("---- SCRIPT ----")
print(script)


# SEO Metadata

In [ ]:
seo = llm(f"""
Create SEO metadata for this YouTube video:
Title, Description, 10 Tags, and 5 Hashtags.
Topic: {chosen_idea}
""")

print("---- SEO METADATA ----")
print(seo)


# Thumbnail Generator (Image Creation)

In [ ]:
from PIL import Image, ImageDraw, ImageFont
import matplotlib.pyplot as plt

def create_thumbnail(title="My Title", subtitle="My Subtitle"):
    # Canvas size
    W, H = 1280, 720
    img = Image.new("RGB", (W, H), "#1A1448")
    draw = ImageDraw.Draw(img)

    font = ImageFont.load_default()

    # --------------------------
    # FUNCTION: Convert text → big image
    # --------------------------
    def create_big_text(text, scale, color, outline):
        temp = Image.new("RGBA", (600, 150), (0, 0, 0, 0))
        tdraw = ImageDraw.Draw(temp)

        # Outline
        for dx in range(-2, 3):
            for dy in range(-2, 3):
                tdraw.text((20+dx, 20+dy), text, font=font, fill=outline)

        # Fill
        tdraw.text((20, 20), text, font=font, fill=color)

        # Scale UP
        big = temp.resize((temp.width * scale, temp.height * scale), Image.NEAREST)
        return big

    # Create BIG title
    big_title = create_big_text(title.upper(), scale=12, color="white", outline="black")

    # Subtitle
    big_sub = create_big_text(subtitle, scale=8, color="yellow", outline="black")

    # Paste Title
    img.paste(big_title, (60, 60), big_title)

    # Add Emoji (RIGHT SIDE of Title)
    draw.text((60 + big_title.width + 40, 120), "🍲", font=font, fill="white")

    # Paste Subtitle
    img.paste(big_sub, (80, 380), big_sub)

    # ----------------------------------------
    # SAVE IMAGE (GitHub will display this file)
    # ----------------------------------------
    img.save("thumbnail_output.png")
    print("✔ Saved as thumbnail_output.png")

    # ----------------------------------------
    # SHOW IMAGE (Will also embed inside notebook)
    # ----------------------------------------
    plt.figure(figsize=(12, 6))
    plt.imshow(img)
    plt.axis("off")
    plt.show()

# Run the function with static text
create_thumbnail("Food Magic", "Delicious Recipes")



# Full Pipeline Output

In [ ]:
topic = input("Enter your topic: ")

full_output = {
    "topic": topic,
    "chosen_idea": chosen_idea,
    "script": script,
    "seo_metadata": seo,
    "thumbnail_file": "thumbnail.png"
}

print(json.dumps(full_output, indent=4))


# Save as JSON File

In [ ]:
with open("youtube_agent_output.json", "w") as f:
    json.dump(full_output, f, indent=4)

print("Saved to youtube_agent_output.json")
